In [1]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Masking,Bidirectional, LSTM, RepeatVector, Dense, TimeDistributed,MaxPooling1D, Flatten, Conv1D,Conv2D,Dropout, MaxPooling2D, GRU # for creating layers inside the Neural Network
from keras.optimizers import Adam , SGD
from keras import metrics
from keras.utils import plot_model
# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version
import numpy.ma as ma
# Sklearn
import sklearn
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.preprocessing import MinMaxScaler # for feature scaling
from sklearn.model_selection import train_test_split
# Visualization
import plotly 
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version
from matplotlib import pyplot as plt

#file accessing
import os
# time stuff
from datetime import timedelta
import calendar


c:\Python\Python310\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


Tensorflow/Keras: 2.9.0
pandas: 1.4.2
numpy: 1.22.4
sklearn: 1.1.1
plotly: 5.9.0


In [2]:
df=pd.read_csv('InSAR_data_south/displacement/export_dataframe1.csv')
df=df.set_index([df.columns[0],df.columns[1]])
df.columns=pd.to_datetime(df.columns, format='%Y%m%d')
df=df.dropna(axis=0, how='all')#drop full nan rows
df

2014-11-08  2014-12-02  2014-12-26  2015-02-12  \
Longitude Latitude                                                   
-120.9146 35.4470          0.0    -13.5180    0.747460      5.0862   
          35.4490          0.0    -13.6410    0.063185      4.4150   
          35.4510          0.0    -13.6300    0.400600      4.7172   
          35.4530          0.0    -13.5690   -0.500240      4.0645   
          35.4550          0.0    -13.5060   -1.107300      2.9022   
...                        ...         ...         ...         ...   
-117.6414 37.6236          0.0      4.8278   -1.306500         NaN   
          37.6256          0.0      4.8698   -1.213700         NaN   
          37.6276          0.0      4.8693   -1.189300         NaN   
          37.6296          0.0      4.8277   -1.190600         NaN   
          37.6316          0.0      4.8729   -1.120000         NaN   

                    2015-03-08  2015-04-01  2015-04-25  2015-05-19  \
Longitude Latitude                                                   
-120.9146 35.4470      -1.9668     1.29470     -5.1067     0.87440   
          35.4490      -2.5803     0.74781     -5.7172     0.19724   
          35.4510      -2.3308     0.99271     -5.4875     0.39402   
          35.4530      -2.8942     0.21585     -6.1191     0.00391   
          35.4550      -4.1777    -1.13510     -7.3587    -1.35330   
...                        ...         ...         ...         ...   
-117.6414 37.6236      -4.1515    -1.99990      9.9914    -3.10550   
          37.6256      -4.1588    -1.73720     10.4130    -2.95840   
          37.6276      -4.1632    -1.71070     10.4830    -2.93180   
          37.6296      -4.1240    -1.89490     10.1870    -3.01790   
          37.6316      -4.1457    -1.70330     10.5180    -2.92690   

                    2015-06-12  2015-07-06  ...  2018-10-18  2018-10-30  \
Longitude Latitude                          ...                           
-120.9146 35.4470     -5.15400     2.55900  ...     -6.5221     -3.3514   
          35.4490     -5.74300     1.93390  ...     -7.3692     -4.3340   
          35.4510     -5.48650     2.15280  ...     -6.8486     -4.0115   
          35.4530     -5.82860     1.83380  ...     -5.6745     -3.0299   
          35.4550     -7.29700     0.46061  ...     -7.9927     -5.4295   
...                        ...         ...  ...         ...         ...   
-117.6414 37.6236     -0.32799     6.98990  ...      8.5800     11.3410   
          37.6256     -0.33838     7.34020  ...      9.2515     11.9960   
          37.6276     -0.34454     7.39780  ...      8.8321     11.5700   
          37.6296     -0.31614     7.12450  ...     10.3320     13.0730   
          37.6316     -0.33153     7.38390  ...      9.3376     12.0530   

                    2018-11-11  2018-11-17  2018-11-23  2018-12-05  \
Longitude Latitude                                                   
-120.9146 35.4470      -4.1662     -6.3219     -11.361     -9.6235   
          35.4490      -5.1182     -7.1733     -12.311    -10.5640   
          35.4510      -4.7786     -6.7119     -11.929    -10.1480   
          35.4530      -3.7675     -5.5943     -10.838     -9.0944   
          35.4550      -6.2580     -7.9820     -13.136    -11.3610   
...                        ...         ...         ...         ...   
-117.6414 37.6236       5.5704      7.6113      17.002      9.6014   
          37.6256       6.0621      8.2387      17.526      9.5942   
          37.6276       5.5897      7.8226      17.066      9.2927   
          37.6296       7.2137      9.3699      18.582     10.8610   
          37.6316       6.0863      8.3323      17.499      9.8190   

                    2018-12-17  2018-12-29  2019-01-10  2019-01-22  
Longitude Latitude                                                  
-120.9146 35.4470      -4.9249     -3.5976     -5.1346     -4.4193  
          35.4490      -5.7429     -4.3201     -5.8205     -5.0144  
          35.4510      -5.1576     

In [3]:
df2=pd.read_csv('InSAR_data_south/displacement/groundwater.csv')
df2=df2.set_index([df2.columns[0],df2.columns[1]])
df2.columns=pd.to_datetime(df2.columns, format='%Y-%m-%d')
df2=df2.dropna(axis=0, how='all')#drop full nan rows
df2

,,2016-01-14,2016-01-26,2016-02-07,2016-02-19,2016-03-02,2016-03-14,2016-03-26,2016-04-07,2016-04-19,2016-05-01,...,2018-10-18,2018-10-30,2018-11-11,2018-11-17,2018-11-23,2018-12-05,2018-12-17,2018-12-29,2019-01-10,2019-01-22
Longitude,Latitude,,,,,,,,,,,,,,,,,,,,,
-120.9486,37.6316,NaN,NaN,NaN,81.0,80.992509,80.970037,80.947566,80.925094,80.902622,80.880150,...,75.653992,75.813688,75.973384,76.0,NaN,NaN,NaN,NaN,NaN,NaN
-120.8926,36.7114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.248964,72.314249,72.379534,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-120.8846,36.6994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.248964,72.314249,72.379534,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-120.8746,36.6874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.248964,72.314249,72.379534,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-120.1368,35.6991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,141.123387,141.263710,141.404032,...,150.898190,151.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-119.0351,36.3933,NaN,NaN,NaN,NaN,NaN,17.409091,17.436364,17.463636,17.490909,17.518182,...,16.978182,17.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-119.0271,36.3793,NaN,NaN,NaN,NaN,NaN,17.409091,17.436364,17.463636,17.490909,17.518182,...,16.978182,17.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-119.0571,36.4493,NaN,NaN,NaN,NaN,NaN,17.409091,17.436364,17.463636,17.490909,17.518182,...,16.978182,17.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
dftest2=df.iloc[df.index.get_level_values('Longitude')< -118.9 ]
dftest2=dftest2.iloc[dftest2.index.get_level_values('Longitude')> -120.6]
dftest2=dftest2.iloc[dftest2.index.get_level_values('Latitude')> 35.3]
dftest2=dftest2.iloc[dftest2.index.get_level_values('Latitude')< 37.3]

In [5]:
for row in range(len(dftest2.index)-1,-1,-1):
    if dftest2.index[row] not in df2.index:
            dftest2.drop(index=dftest2.index[row], inplace=True)
dftest2

2014-11-08  2014-12-02  2014-12-26  2015-02-12  \
Longitude Latitude                                                   
-120.5987 36.8834          0.0     -7.7869     2.01640      4.0537   
          36.9674          0.0     -8.1783    -0.73803      1.3032   
-120.5967 36.6013          0.0     -7.9023     0.44939      1.3560   
          36.7694          0.0     -8.1188     0.99818      1.8494   
          36.8514          0.0     -7.4329     1.14450      2.6733   
...                        ...         ...         ...         ...   
-118.9071 35.3750          0.0     -3.8828    -1.19990     -1.2152   
-118.9051 35.3430          0.0     -4.1356    -1.12550     -1.2232   
-118.9031 35.3110          0.0     -4.1879    -1.12130     -1.7977   
          35.4270          0.0     -4.1305    -1.29110     -1.1049   
-118.9011 35.3950          0.0     -4.1354    -1.32910     -1.1547   

                    2015-03-08  2015-04-01  2015-04-25  2015-05-19  \
Longitude Latitude                                                   
-120.5987 36.8834      2.06420      4.8085     0.57137      3.0914   
          36.9674     -0.97954      2.5810    -1.72690      1.7493   
-120.5967 36.6013     -0.54627      3.5450     0.12322      3.3109   
          36.7694      0.35231      3.7629    -0.50686      2.4502   
          36.8514      1.04980      4.3802    -0.36974      2.8307   
...                        ...         ...         ...         ...   
-118.9071 35.3750     -2.12550     -3.4199    -0.94401     -3.6226   
-118.9051 35.3430     -2.18530     -3.3244    -0.70324     -3.5807   
-118.9031 35.3110     -2.28030     -3.4793    -0.84339     -3.6445   
          35.4270     -2.53270     -3.4353    -0.79675     -3.7570   
-118.9011 35.3950     -2.29250     -3.5912    -0.82694     -3.6488   

                    2015-06-12  2015-07-06  ...  2018-10-18  2018-10-30  \
Longitude Latitude                          ...                           
-120.5987 36.8834      4.13000     0.25740  ...    -7.29210   -4.124700   
          36.9674      2.54750    -1.49550  ...   -12.25300   -9.062500   
-120.5967 36.6013      5.71110     2.50060  ...    -1.81620    1.239700   
          36.7694      5.17660     0.26020  ...    -3.32390    0.210520   
          36.8514      4.21690    -0.27127  ...    -3.82210   -0.593180   
...                        ...         ...  ...         ...         ...   
-118.9071 35.3750      0.56910    -1.45060  ...     2.44230    2.689300   
-118.9051 35.3430      0.53279    -1.50250  ...     1.45350    1.393300   
-118.9031 35.3110      0.30264    -2.15690  ...    -0.17429    0.098737   
          35.4270      0.58227    -1.29810  ...     3.02090    3.222600   
-118.9011 35.3950      0.71169    -1.05600  ...     2.84470    3.160200   

                    2018-11-11  2018-11-17  2018-11-23  2018-12-05  \
Longitude Latitude                                                   
-120.5987 36.8834     -4.42790    -5.23150    -8.96810   -8.516600   
          36.9674    -10.04000   -10.74100   -14.23000  -13.568000   
-120.5967 36.6013      0.69477     0.47607    -3.24070   -2.944300   
          36.7694     -0.38331    -1.39020    -5.40670   -4.185600   
          36.8514     -1.01480    -1.57170    -5.77840   -4.350000   
...                        ...         ...         ...         ...   
-118.9071 35.3750      3.37490     3.22130     3.32230    2.285500   
-118.9051 35.3430      2.12930     1.83520     2.14240    1.547400   
-118.9031 35.3110      0.56824     0.16814     0.69418   -0.011814   
          35.4270      4.33210     3.71760     4.08800    2.244900   
-118.9011 35.3950      4.13480     3.61790     3.82230    2.624100   

                    2018-12-17  2018-12-29  2019-01-10  2019-01-22  
Longitude Latitude                                                  
-120.5987 36.8834     -5.29430    -4.32170    -4.43320    -1.74300  
          36.9674    -11.84400   -10.87200   -11.07400    -8.54130  
-120.5967 36.6013      0.52726     

In [6]:
dftest2.to_csv('shorterMainFile.csv')